<a href="https://colab.research.google.com/github/josorio398/ALGEBRA-LINEAL-CON-PYTHON/blob/master/Librer%C3%ADa_de_modelos_de_solubilidad_Modelo_Apelblat_Modificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

<FONT SIZE=6><strong>ANÁLISIS COMPUTACIONAL: ECUACIÓN DE APELBLAT MODIFICADA

$$\ln(x) = A + \dfrac{B}{T} + C \cdot \ln T$$

In [ ]:
from pandas.core.indexes import extension
import pandas as pd
import numpy as np
from IPython.display import display, Math, Latex, Markdown,HTML
from IPython.core.display import display, HTML
from scipy.optimize import curve_fit
import plotly.graph_objects as go
from google.colab import files



def func(T,A,B,C):
    return np.exp(A + B/T + C*np.log(T))

class dataframe:
    
    def __init__(self,url):
        self.url =url
    
    @property
    def value_temp(self):
        df = pd.read_csv(self.url)
        temp = pd.DataFrame(df["T"])
        return temp
        
    @property
    def show(self):
        df = pd.read_csv(self.url)
        return df

# CLASE PARA EL MODELO APELBLAT MODIFICADO

class modified_apelblat_model(dataframe):
    
    @property
    def equation(self):
        salida = display(HTML('<h2> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Modified Apelblat Equation</h2>'))
        display(Math(r'$$\ln(x)=A + \dfrac{B}{T} + C\ln{T}$$'))
        return salida


    def parameters(self, method="lm", maxfev=20000, sd = True):
        
        df = pd.read_csv(self.url)
        W = list(df.columns[1:])

        para_A = []
        para_B = []
        para_C = []

        desv_para_A = []
        desv_para_B = []
        desv_para_C = []

        for i in W:
            xdat = df["T"].values
            ydat = (df[i].values)
            popt,mcov = curve_fit(func,xdat,ydat,method=method ,maxfev=maxfev)

            para_A.append(popt[0])
            para_B.append(popt[1])
            para_C.append(popt[2])

            desv_para_A.append(np.sqrt((np.diag(mcov))[0]))
            desv_para_B.append(np.sqrt((np.diag(mcov))[1]))
            desv_para_C.append(np.sqrt((np.diag(mcov))[2]))

        if sd == True:
            DF = pd.DataFrame({"x":df.columns[1:],'A':para_A,"σ_A":desv_para_A,'B':para_B,"σ_B":desv_para_B,'C':para_C,"σ_C":desv_para_C})
        else:
            DF = pd.DataFrame({"x":df.columns[1:],'A':para_A,'B':para_B,'C':para_C})
        return  DF
    
    
    def values_exp(self,temp = True):

        df = pd.read_csv(self.url)
        W = list(df.columns[1:])

        C_temp = []
        C_exp  = []
       
        for i in W:

            xdat = df["T"].values
            Xdat = xdat.tolist()

            ydat = (df[i].values)
            Ydat =  ydat.tolist()

            C_temp += Xdat
            C_exp  += Ydat
 
        arr_tem = np.array(C_temp)
        arr_exp = np.array(C_exp)

        if temp == True: 
            data_frame = pd.DataFrame({'Temperatura': arr_tem,'Xexp':arr_exp})
        else : 
            data_frame = pd.DataFrame({'Xexp':arr_exp})
            
        return data_frame


    def values_cal(self,method="lm", maxfev=20000,temp = True, RD = False):

        df = pd.read_csv(self.url)
        W  = list(df.columns[1:])

        para_A = []
        para_B = []
        para_C = []


        C_temp = []
        C_cal  = []
        C_RD   = []

        for i in W:
            xdat = df["T"].values
            ydat = (df[i].values)
            popt,mcov = curve_fit(func,xdat,ydat,method=method,maxfev=maxfev)

            para_A.append(popt[0])
            para_B.append(popt[1])
            para_C.append(popt[2])

        for i in W:
            xdat = df["T"].values
            Xdat = xdat.tolist()

            ydat = (df[i].values)
            Ydat =  ydat.tolist()

            ymodel = (func(xdat,para_A[W.index(i)],para_B[W.index(i)],para_C[W.index(i)]))
            Ymodel = ymodel.tolist()

            Relative_differences = ((ydat - ymodel)/ydat).tolist()


            C_temp += Xdat
            C_cal  += Ymodel
            C_RD   += Relative_differences
 

        arr_tem = np.array(C_temp)
        arr_cal = np.array(C_cal)
        arr_RD  = np.array(C_RD )
        
        if  temp == True  and RD == False:
            data_frame = pd.DataFrame({'Temperatura': arr_tem,'Xcal':arr_cal})
        elif temp == True and RD == True:
            data_frame = pd.DataFrame({'Temperatura': arr_tem,'Xcal':arr_cal,'RD':arr_RD})
        elif temp == False and RD == False:
            data_frame = pd.DataFrame({'Xcal':arr_cal})
        elif temp == False and RD == True:
            data_frame = pd.DataFrame({'Xcal':arr_cal,'RD':arr_RD})
        return data_frame

    def values(self,method="lm", maxfev=20000, temp = True):

        df = pd.read_csv(self.url)
        W  = list(df.columns[1:])

        para_A = []
        para_B = []
        para_C = []

        C_temp = []
        C_exp  = []
        C_cal  = []
        C_RD   = []

        for i in W:
            xdat = df["T"].values
            ydat = (df[i].values)
            popt,mcov = curve_fit(func,xdat,ydat,method=method,maxfev=maxfev)

            para_A.append(popt[0])
            para_B.append(popt[1])
            para_C.append(popt[2])


        for i in W:
  
            xdat = df["T"].values
            Xdat = xdat.tolist()

            ydat = (df[i].values)
            Ydat =  ydat.tolist()

            ymodel = (func(xdat,para_A[W.index(i)],para_B[W.index(i)],para_C[W.index(i)]))
            Ymodel = ymodel.tolist()

            RD = ((ydat - ymodel)/ydat).tolist()


            C_temp += Xdat
            C_exp  += Ydat
            C_cal  += Ymodel
            C_RD   += RD
 

        arr_tem = np.array(C_temp)
        arr_exp = np.array(C_exp)
        arr_cal = np.array( C_cal)
        arr_RD  = np.array( C_RD )

        if temp == True: 
            data_frame = pd.DataFrame({'Temperatura': arr_tem,'Xexp':arr_exp,'Xcal':arr_cal,'RD':arr_RD})
        else:
            data_frame = pd.DataFrame({'Xexp':arr_exp,'Xcal':arr_cal,'RD':arr_RD})

        return data_frame

    def statistic_MAPE(self,method="lm", maxfev=20000, opt= "all"):


        df = pd.read_csv(self.url)
        W  = list(df.columns[1:])

        para_A = []
        para_B = []
        para_C = []

        C_temp = []
        C_exp  = []
        C_cal  = []
        C_RD   = []

        for i in W:
            xdat = df["T"].values
            ydat = (df[i].values)
            popt,mcov = curve_fit(func,xdat,ydat,method=method,maxfev=maxfev)

            para_A.append(popt[0])
            para_B.append(popt[1])
            para_C.append(popt[2])


        for i in W:
  
            xdat = df["T"].values
            Xdat = xdat.tolist()

            ydat = (df[i].values)
            Ydat =  ydat.tolist()

            ymodel = (func(xdat,para_A[W.index(i)],para_B[W.index(i)],para_C[W.index(i)]))
            Ymodel = ymodel.tolist()

            RD = ((ydat - ymodel)/ydat).tolist()


            C_temp += Xdat
            C_exp  += Ydat
            C_cal  += Ymodel
            C_RD   += RD
 

        arr_tem = np.array( C_temp)
        arr_exp = np.array( C_exp)
        arr_cal = np.array( C_cal)
        arr_RD  = np.array( C_RD )

        data_frame = pd.DataFrame({'Temperatura': arr_tem,'Xexp':arr_exp,'Xcal':arr_cal,'RD':arr_RD})

        
        MAPES = []
        INDEX = []

        Temp = df["T"].values

        for i in range(len(Temp)):
            df_mask = data_frame['Temperatura'] == Temp[i]
            data_filter = data_frame[df_mask]
            MAPE = sum(abs(data_filter["RD"]))*100/len(data_filter['Temperatura'])
            MAPES.append(MAPE)
            INDEX.append("T"+str(i+1))


        if opt == "all": 

            data_mape = print("Mean Absolute Percentage Error, MAPE =",sum(MAPES)/len(Temp))
        else:
            data_mape = pd.DataFrame({"T":INDEX,"Temp":Temp,"MAPE":MAPES})

        return data_mape

    def statistic_RMSD(self,method="lm", maxfev=20000):


        df = pd.read_csv(self.url)
        W  = list(df.columns[1:])

        para_A = []
        para_B = []
        para_C = []

        C_temp = []
        C_exp  = []
        C_cal  = []
        C_RD   = []

        for i in W:
            xdat = df["T"].values
            ydat = (df[i].values)
            popt,mcov = curve_fit(func,xdat,ydat,method=method,maxfev=maxfev)

            para_A.append(popt[0])
            para_B.append(popt[1])
            para_C.append(popt[2])


        for i in W:
  
            xdat = df["T"].values
            Xdat = xdat.tolist()

            ydat = (df[i].values)
            Ydat =  ydat.tolist()

            ymodel = (func(xdat,para_A[W.index(i)],para_B[W.index(i)],para_C[W.index(i)]))
            Ymodel = ymodel.tolist()

            RD = ((ydat - ymodel)/ydat).tolist()


            C_temp += Xdat
            C_exp  += Ydat
            C_cal  += Ymodel
            C_RD   += RD
 

        arr_tem = np.array( C_temp)
        arr_exp = np.array( C_exp)
        arr_cal = np.array( C_cal)
        

        ss_res = np.sum((arr_cal - arr_exp)**2 )
        RMSD = (np.sqrt(ss_res/len(arr_exp)))

        return print("Root Mean Square Deviation, RMSD =",RMSD )


    def statistic_AIC(self,method="lm", maxfev=20000, opt ="corrected"):


        df = pd.read_csv(self.url)
        W  = list(df.columns[1:])

        para_A = []
        para_B = []
        para_C = []

        C_temp = []
        C_exp  = []
        C_cal  = []
        C_RD   = []

        for i in W:
            xdat = df["T"].values
            ydat = (df[i].values)
            popt,mcov = curve_fit(func,xdat,ydat,method=method,maxfev=maxfev)

            para_A.append(popt[0])
            para_B.append(popt[1])
            para_C.append(popt[2])


        for i in W:
  
            xdat = df["T"].values
            Xdat = xdat.tolist()

            ydat = (df[i].values)
            Ydat =  ydat.tolist()

            ymodel = (func(xdat,para_A[W.index(i)],para_B[W.index(i)],para_C[W.index(i)]))
            Ymodel = ymodel.tolist()

            RD = ((ydat - ymodel)/ydat).tolist()


            C_temp += Xdat
            C_exp  += Ydat
            C_cal  += Ymodel
            C_RD   += RD
 

        arr_tem = np.array( C_temp)
        arr_exp = np.array( C_exp)
        arr_cal = np.array( C_cal)
        arr_RD  = np.array(C_RD )
        
        k = 3
        N =len(arr_RD)
        ss_res = np.sum((arr_cal - arr_exp)**2 )
        AIC = N*np.log(ss_res/N)+2*k
        
        AICc = abs(AIC +((2*k**2+2*k)/(N-k-1)))
        
        if opt =="corrected":
            aic = print("Akaike Information Criterion corrected , AICc =",AICc)
        else:
            aic = print("Akaike information criterion, AIC =",AIC )
        return aic

    def statistic_RS(self,method="lm", maxfev=20000):


        df = pd.read_csv(self.url)
        W  = list(df.columns[1:])

        para_A = []
        para_B = []
        para_C = []

        C_temp = []
        C_exp  = []
        C_cal  = []
        C_RD   = []

        for i in W:
            xdat = df["T"].values
            ydat = (df[i].values)
            popt,mcov = curve_fit(func,xdat,ydat,method=method,maxfev=maxfev)

            para_A.append(popt[0])
            para_B.append(popt[1])
            para_C.append(popt[2])


        for i in W:
  
            xdat = df["T"].values
            Xdat = xdat.tolist()

            ydat = (df[i].values)
            Ydat =  ydat.tolist()

            ymodel = (func(xdat,para_A[W.index(i)],para_B[W.index(i)],para_C[W.index(i)]))
            Ymodel = ymodel.tolist()

            RD = ((ydat - ymodel)/ydat).tolist()


            C_temp += Xdat
            C_exp  += Ydat
            C_cal  += Ymodel
            C_RD   += RD
 

        arr_tem = np.array( C_temp)
        arr_exp = np.array( C_exp)
        arr_cal = np.array( C_cal)
        arr_RD  = np.array(C_RD )
       
        ss_res = np.sum((arr_cal - arr_exp)**2 )
        ss_tot = np.sum( (arr_exp - np.mean(arr_exp) )**2  )

        R2 = 1 - (ss_res / ss_tot)
            
        return print("Coefficient of Determination, R^2 =",R2)

    def statistics(self,method="lm", maxfev=20000):


        df = pd.read_csv(self.url)
        W  = list(df.columns[1:])

        para_A = []
        para_B = []
        para_C = []

        C_temp = []
        C_exp  = []
        C_cal  = []
        C_RD   = []

        for i in W:
            xdat = df["T"].values
            ydat = (df[i].values)
            popt,mcov = curve_fit(func,xdat,ydat,method=method,maxfev=maxfev)

            para_A.append(popt[0])
            para_B.append(popt[1])
            para_C.append(popt[2])


        for i in W:
  
            xdat = df["T"].values
            Xdat = xdat.tolist()

            ydat = (df[i].values)
            Ydat =  ydat.tolist()

            ymodel = (func(xdat,para_A[W.index(i)],para_B[W.index(i)],para_C[W.index(i)]))
            Ymodel = ymodel.tolist()

            RD = ((ydat - ymodel)/ydat).tolist()


            C_temp += Xdat
            C_exp  += Ydat
            C_cal  += Ymodel
            C_RD   += RD
 

        arr_tem = np.array( C_temp)
        arr_exp = np.array( C_exp)
        arr_cal = np.array( C_cal)
        arr_RD  = np.array(C_RD )

        MAPE = sum(abs(arr_RD))*100/len(arr_RD)

        ss_res = np.sum((arr_cal - arr_exp)**2 )
        ss_tot = np.sum( (arr_exp - np.mean(arr_exp) )**2  )

        R2 = 1 - (ss_res / ss_tot)

        ss_res = np.sum((arr_cal - arr_exp)**2 )
        RMSD = (np.sqrt(ss_res/len(arr_exp)))


        k = 3
        N =len(arr_RD)
        ss_res = np.sum((arr_cal - arr_exp)**2 )
        AIC = N*np.log(ss_res/N)+2*k
        AICc = abs(AIC +((2*k**2+2*k)/(N-k-1)))

        ss_res = np.sum((arr_cal - arr_exp)**2 )
        ss_tot = np.sum( (arr_exp - np.mean(arr_exp) )**2  )

        R2 = 1 - (ss_res / ss_tot)

        col1 = np.array(["MAPE","RMSD","AICc","R^2"])
        col2 = np.array([MAPE,RMSD,AICc,R2])
        estadis = pd.DataFrame({"statistics":col1,"values":col2})
            
        return estadis

    def summary_data(self,method="lm", maxfev=20000, temp = True, sd =True,name= "modified_apelblat"):
        
        nombre = name
        DATA = pd.concat([self.values(method = method, maxfev = maxfev, temp = temp),self.statistic_MAPE(method = method, maxfev = maxfev,opt="temp"),self.parameters(method = method, maxfev = maxfev, sd = sd ),self.statistics(method = method, maxfev = maxfev)], axis=1)
        
        
        url_1 = "/content/sample_data/"+ nombre +".xlsx"
        url_3 = "/content/sample_data/"+ nombre +".csv"
        DATA.to_excel(url_1,sheet_name='Modified_Apelblat')
        DATA.to_csv(url_3)
        return DATA

    def plot(self,method="lm", maxfev=20000, name = "modified_apelblat" ):

        nombre= name
        
        url_2 = "/content/sample_data/"+ nombre+".pdf"

        df = pd.read_csv(self.url)
        W  = list(df.columns[1:])

        para_A = []
        para_B = []
        para_C = []

        C_temp = []
        C_exp  = []
        C_cal  = []
        C_RD   = []

        for i in W:
            xdat = df["T"].values
            ydat = (df[i].values)
            popt,mcov = curve_fit(func,xdat,ydat,method=method,maxfev=maxfev)

            para_A.append(popt[0])
            para_B.append(popt[1])
            para_C.append(popt[2])


        for i in W:
  
            xdat = df["T"].values
            Xdat = xdat.tolist()

            ydat = (df[i].values)
            Ydat =  ydat.tolist()

            ymodel = (func(xdat,para_A[W.index(i)],para_B[W.index(i)],para_C[W.index(i)]))
            Ymodel = ymodel.tolist()

            RD = ((ydat - ymodel)/ydat).tolist()


            C_temp += Xdat
            C_exp  += Ydat
            C_cal  += Ymodel
            C_RD   += RD
 

        arr_tem = np.array( C_temp)
        arr_exp = np.array( C_exp)
        arr_cal = np.array( C_cal)
        arr_RD  = np.array(C_RD )

        fig = go.Figure()


        numerofilas = len(df["T"])
        numerocolumnas = len(df.columns )-2
        L = [numerofilas*i for i in range(numerocolumnas+2)]

        X = np.linspace(min(arr_cal),max(arr_exp),200)

        for i in range(11):
            fig.add_trace(go.Scatter(x=arr_cal[L[i]:L[i+1]], y=arr_exp[L[i]:L[i+1]],name=" w"+str(i) ,mode='markers',marker=dict(size=6,line=dict(width=0.5,color='DarkSlateGrey'))))


        fig.add_trace(go.Scatter(x=X,y=X,name="$x_{exp}=x_{cal}$"))


        fig.update_xaxes(title = "$x_{cal}$", title_font=dict(size=30, family='latex', color='rgb(1,21,51)'))
        fig.update_yaxes(title = "$x_{exp}$ ",title_font=dict(size=30,family='latex', color='rgb(1,21,51)'))
        fig.update_layout(title='',title_font=dict(size=26, family='latex', color= "rgb(1,21,51)"),width=1010, height=550)
        fig.update_layout(legend=dict(orientation="h",y=1.2,x=0.03),title_font=dict(size=40, color='rgb(1,21,51)'))
        fig.write_image(url_2)
        return fig.show()

    
    
    def summary_download(self,name = "modified_apelblat",ext= "xlsx"):
        nombre = name
        extension = ext
        url_1 = "/content/sample_data/"+ nombre +"."+extension
        return files.download(url_1)

    def plot_download(self,name = "modified_apelblat"):
        nombre = name
        url_2 = "/content/sample_data/"+ nombre+".pdf"
        return files.download(url_2)



In [ ]:
datos = "https://raw.githubusercontent.com/josorio398/Datos-Experimentales-Articulos/main/Datos-Experimentales-Progesterona.csv"
tabla=dataframe(datos)
tabla.show

,T,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
0,278.15,0.003474,0.004135,0.004902,0.005776,0.006768,0.008212,0.009506,0.011551,0.013446,0.016160,0.019426
1,283.15,0.004699,0.005635,0.006558,0.007875,0.009291,0.010839,0.012840,0.015361,0.017629,0.021631,0.025138
2,288.15,0.006434,0.007651,0.008837,0.010590,0.012285,0.014382,0.017283,0.020078,0.023767,0.027730,0.032794
3,293.15,0.008758,0.010016,0.011825,0.014066,0.016600,0.019189,0.022561,0.026279,0.030989,0.035827,0.041767
4,298.15,0.011372,0.013420,0.015643,0.018082,0.021239,0.025000,0.028850,0.033605,0.038841,0.045945,0.054639
5,303.15,0.015176,0.017438,0.020472,0.023678,0.027654,0.032454,0.037721,0.043093,0.050664,0.057556,0.068107
6,308.15,0.019611,0.022677,0.026564,0.030550,0.035239,0.040859,0.047857,0.054909,0.064172,0.074461,0.086796
7,313.15,0.025689,0.029450,0.033628,0.039366,0.044885,0.051327,0.060326,0.070076,0.079765,0.091857,0.107922
8,318.15,0.032771,0.038110,0.043439,0.050537,0.058350,0.067012,0.075074,0.086158,0.101364,0.115677,0.132370


In [ ]:
modelo = modified_apelblat_model(datos)
modelo.equation

<IPython.core.display.Math object>

In [ ]:
parametros = modelo.parameters(method="lm", maxfev=10000, sd = True)
parametros 

,x,A,σ_A,B,σ_B,C,σ_C
0,x0,9.960123,27.989403,-4876.278485,1269.499292,0.338382,4.165043
1,x1,-62.818626,18.248950,-1549.587683,827.344193,11.179276,2.715789
2,x2,5.339244,31.389973,-4534.927294,1422.895357,1.002253,4.671546
3,x3,-56.180529,19.848951,-1718.278334,899.390967,10.168339,2.954175
4,x4,-70.553185,47.936802,-1005.402970,2171.369831,12.297885,7.134978
5,x5,-57.387980,64.980455,-1522.053808,2942.513316,10.318684,9.672271
6,x6,71.023216,19.740180,-7199.661997,893.855062,-8.847144,2.938333
7,x7,29.135895,35.005212,-5253.764058,1584.434909,-2.615128,5.210892
8,x8,-12.546319,39.457567,-3361.652155,1785.634090,3.613011,5.873851
9,x9,-31.078202,34.185389,-2435.399382,1546.288868,6.347004,5.089432


In [ ]:
valores_experimentales = modelo.values_exp(temp=True)
valores_experimentales

,Temperatura,Xexp
0,278.15,0.003474
1,283.15,0.004699
2,288.15,0.006434
3,293.15,0.008758
4,298.15,0.011372
...,...,...
94,298.15,0.054639
95,303.15,0.068107
96,308.15,0.086796
97,313.15,0.107922


In [ ]:
valores_calculados = modelo.values_cal(method="lm", maxfev=10000,temp = True, RD = True)
valores_calculados

,Temperatura,Xcal,RD
0,278.15,0.003460,0.004203
1,283.15,0.004744,-0.009443
2,288.15,0.006434,0.000031
3,293.15,0.008637,0.013854
4,298.15,0.011481,-0.009618
...,...,...,...
94,298.15,0.054264,0.006847
95,303.15,0.068764,-0.009652
96,308.15,0.086344,0.005205
97,313.15,0.107480,0.004094


In [ ]:
valores= modelo.values(method="lm", maxfev=10000,temp = True)
valores

,Temperatura,Xexp,Xcal,RD
0,278.15,0.003474,0.003460,0.004203
1,283.15,0.004699,0.004744,-0.009443
2,288.15,0.006434,0.006434,0.000031
3,293.15,0.008758,0.008637,0.013854
4,298.15,0.011372,0.011481,-0.009618
...,...,...,...,...
94,298.15,0.054639,0.054264,0.006847
95,303.15,0.068107,0.068764,-0.009652
96,308.15,0.086796,0.086344,0.005205
97,313.15,0.107922,0.107480,0.004094


In [ ]:
mapes = modelo.statistic_MAPE(method="lm", maxfev=20000, opt= "temp")
mapes

,T,Temp,MAPE
0,T1,278.15,1.560962
1,T2,283.15,0.878958
2,T3,288.15,0.584439
3,T4,293.15,0.949976
4,T5,298.15,0.803042
5,T6,303.15,0.667359
6,T7,308.15,0.440716
7,T8,313.15,0.758574
8,T9,318.15,0.285202


In [ ]:
mapes = modelo.statistic_MAPE(method="lm", maxfev=20000, opt="all")
mapes

Mean Absolute Percentage Error, MAPE = 0.7699142615606795


In [ ]:
rmsd = modelo.statistic_RMSD(method="lm", maxfev=20000)
rmsd

Root Mean Square Deviation, RMSD = 0.0003035918045333601


In [ ]:
aic = modelo.statistic_AIC(method="lm", maxfev=20000)
aic

Akaike Information Criterion corrected , AICc = 1597.5130168264163


In [ ]:
r2 = modelo.statistic_RS(method="lm", maxfev=20000)
r2

Coefficient of Determination, R^2 = 0.999873403786956


In [ ]:
estadisticos = modelo.statistics(method="lm", maxfev=20000)
estadisticos

,statistics,values
0,MAPE,0.769914
1,RMSD,0.000304
2,AICc,1597.513017
3,R^2,0.999873


In [ ]:
modelo.summary_data()

,Temperatura,Xexp,Xcal,RD,T,Temp,MAPE,x,A,σ_A,B,σ_B,C,σ_C,statistics,values
0,278.15,0.003474,0.003460,0.004203,T1,278.15,1.560962,x0,9.960123,27.989403,-4876.278485,1269.499292,0.338382,4.165043,MAPE,0.769914
1,283.15,0.004699,0.004744,-0.009443,T2,283.15,0.878958,x1,-62.818626,18.248950,-1549.587683,827.344193,11.179276,2.715789,RMSD,0.000304
2,288.15,0.006434,0.006434,0.000031,T3,288.15,0.584439,x2,5.339244,31.389973,-4534.927294,1422.895357,1.002253,4.671546,AICc,1597.513017
3,293.15,0.008758,0.008637,0.013854,T4,293.15,0.949976,x3,-56.180529,19.848951,-1718.278334,899.390967,10.168339,2.954175,R^2,0.999873
4,298.15,0.011372,0.011481,-0.009618,T5,298.15,0.803042,x4,-70.553185,47.936802,-1005.402970,2171.369831,12.297885,7.134978,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,298.15,0.054639,0.054264,0.006847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,303.15,0.068107,0.068764,-0.009652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,308.15,0.086796,0.086344,0.005205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,313.15,0.107922,0.107480,0.004094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
modelo.plot()

In [ ]:
modelo.summary_download(ext = "csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
modelo.plot_download()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

https://stackoverflow.com/questions/54109548/how-to-save-pandas-to-excel-with-different-colors